<a href="https://colab.research.google.com/github/thanit456/NeuroSummary/blob/two/headline_classification_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load preprocessed data

In [0]:
import pickle
import numpy as np
import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tqdm

THAIGOV_PATH = '/content/drive/Shared drives/NeuroSummary/data/b_data_playground/thai_gov_split/'

with open(THAIGOV_PATH + 'train.pkl', 'rb') as f:
  train_df = pickle.load(f)

with open(THAIGOV_PATH + 'val.pkl', 'rb') as f:
  val_df = pickle.load(f)

with open(THAIGOV_PATH + 'test.pkl', 'rb') as f:
  test_df = pickle.load(f)

In [0]:
with open(THAIGOV_PATH + 'train_stop.pkl', 'rb') as f:
  train_stop_df = pickle.load(f)

with open(THAIGOV_PATH + 'val_stop.pkl', 'rb') as f:
  val_stop_df = pickle.load(f)

with open(THAIGOV_PATH + 'test_stop.pkl', 'rb') as f:
  test_stop_df = pickle.load(f)

In [0]:
def create_label(df_headline, df_content):
  labels = []
  for idx in tqdm.tqdm(range(len(df_content))):
    tmp = []
    for word in df_content[idx]:
      if word in df_headline.iloc[idx]:
        tmp.append(1)
      else:
        tmp.append(0)
    labels.append(tmp)
  return labels

In [52]:
train_df.head()

,content,headline,class
8608,"[กรมเจ้าท่า, จท., กระทรวงคมนาคม, อาศัย, อำนาจ,...","[กรมเจ้าท่า, ปรับปรุง, อัตรา, ค่า, โดยสาร, เรื...",ด้านเศรษฐกิจ
8593,"[รอง, นรม, พล.อ.ฉัตรชัยฯ, เป็น, ประธาน, การ, ป...","[รอง, นรม, พล.อ.ฉัตรชัยฯ, เป็น, ประธาน, การ, ป...",ข่าวทำเนียบรัฐบาล
10488,"[พลเอก สุรศักดิ์ กาญจนรัตน์, รัฐมนตรี, ว่าการ,...","[รมว., ทส., เปิด, งาน, ประชารัฐร่วมใจ, คน, ลำ,...",ด้านความมั่นคง
12624,"[เมื่อ, วัน, ที่, 5, กุมภาพันธ์, 2561, เวลา, 1...","[นายก, รัฐมนตรี, ลง, พื้นที่, เยี่ยมชม, วิถี, ...",ด้านเศรษฐกิจ
17242,"[กระทรวงดิจิทัลฯ, กระทรวงดิจิทัล, ฯ, สนับสนุน,...","[กระทรวงดิจิทัลฯ, หนุน, สดช., จับ, มือ, ทีโอที...",ด้านสังคม


# Create dictionary

In [0]:
train_set = train_df 
val_set = val_df
test_set = test_df

In [54]:
## ! use only n first words for headline generation
def use_firt_n_words(df_content, n):
  new_ls = []
  for content in df_content:
    new_ls.append(content[:n])
  return new_ls

words_50_train_contents = use_firt_n_words(train_set['content'], n=50)
words_50_val_contents = use_firt_n_words(val_set['content'], n=50)
words_50_test_contents = use_firt_n_words(test_set['content'], n=50)

train_labels = create_label(train_set['headline'],  words_50_train_contents)
val_labels = create_label(val_set['headline'],  words_50_val_contents)
test_labels = create_label(test_set['headline'],  words_50_test_contents)

100%|██████████| 2271/2271 [00:00<00:00, 2531.47it/s]


In [55]:
import collections
def create_index(input_data,threshold):
    input_text = [data for data in input_data]
    # counts of word type has to be above or equal threshold
    words = [word for sublist in input_text for word in sublist]
    print("words :",words)
    word_count_all = list()
    word_count = list()
    #use set and len to get the number of unique words
    word_count_all.extend(collections.Counter(words).most_common(len(set(words))))
    unkcnt = 0
    for (word,cnt) in word_count_all:
      if cnt >= threshold:
        word_count.append((word,cnt))
      else:
        unkcnt+=cnt
    #include a token for unknown word
    word_count.append(("UNK",unkcnt))
    #print out 10 most frequent words
    print("top 10: ",word_count[:10])
    print("bottom 10: ",word_count[-10:])
    dictionary = dict()
    dictionary["for_keras_zero_padding"] = 0
    for word in word_count:
      dictionary[word[0]] = len(dictionary)
    dictionary['<s>'] = len(dictionary) 
    dictionary['</s>'] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    
    # for data in input_data:
    #   sub_data = list()
    #   sub_label = list()
    #   error_list = [data[i] for i in range(1,len(data))]

    #   for i in range(len(data[0])):
    #     sub_label.append(0)
    #     for error_range in error_list:
    #       if i>=error_range[0] and i<error_range[1]:
    #         sub_label[-1] = 1
    #         break.
    return dictionary, reverse_dictionary
dict_t, rev_dict_t = create_index(input_data=words_50_train_contents,threshold=0)
# dict_stop_t, rev_dict_stop_t = create_index(input_data=words_70_train_stop_contents+words_20_train_stop_headlines,threshold=0)
print('Vocab size (Content): ',len(dict_t))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [56]:
print('Vocab size: ',len(dict_t))

Vocab size:  19627


In [57]:
dict_t['<s>']
list(dict_t.values())[-5:]

[19622, 19623, 19624, 19625, 19626]

In [0]:
def word_to_idx(input_data,dictionary) :
  X = list()
  for data in input_data:
    sub_data = []
    for word in data:
      if word in dictionary:
        sub_data.append(dictionary[word])
      else:
        sub_data.append(dictionary["UNK"])
    X.append(sub_data)
  return np.array(X)

In [0]:
X_train = word_to_idx(words_50_train_contents,dict_t)
y_train = train_labels
X_val = word_to_idx(words_50_val_contents,dict_t)
y_val = val_labels
X_test = word_to_idx(words_50_test_contents,dict_t)
y_test = test_labels

# Data generator

In [0]:
# https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dimX=70,dimY=20, n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dimX = dimX
        self.dimY = dimY
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        label_temp = [self.labels[k] for k in indexes]
        # Generate data
        X, y_shift,y = self.__data_generation(list_IDs_temp,label_temp)
        return [X,y_shift],y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, X,y):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        # X = np.empty((self.batch_size, self.dimX))
        # y = np.empty((self.batch_size, self.dimY))

        # Generate data
        # for i, ID in enumerate(list_IDs_temp):
        #     # Store sample
        #     X[i,] = list_IDs_temp
        
        y = list(map(lambda mem :np.append(mem,[len(dict_t)-1]),y))
        y_shift = list(map(lambda mem :np.append([len(dict_t)-2], mem[:-1]),y))
        X = pad_sequences(X,maxlen=maxlen,padding='post')
        y = pad_sequences(y,maxlen=maxlen_output,padding='post') 
        y_shift = pad_sequences(y_shift,maxlen=maxlen_output,padding='post')
        # X = np.array(keras.utils.to_categorical(X, num_classes=self.n_classes), dtype=object)
        y = np.array(keras.utils.to_categorical(y, num_classes=self.n_classes))
        y_shift = np.array(keras.utils.to_categorical(y_shift, num_classes=self.n_classes))
        return X, y_shift,y

# Modelling

In [0]:
maxlen = 50 
maxlen_output = 20 # <s>ABC and ABC</s>
vocab_size = len(dict_t)
# output_vocab_size = len(dict_t)
m=15000
Tx=maxlen
Ty=maxlen_output

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply,Add,Conv1D,GRU,TimeDistributed
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda, Embedding,concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, TensorBoard
import tensorflow.keras.backend as K

import numpy as np
from datetime import datetime

In [0]:
# # from my_classes import DataGenerator

# # Parameters
# params = {'dimX': maxlen,
#           'dimY': maxlen_output,
#           'batch_size': 512,
#           'n_classes': vocab_size,
#           'n_channels': 1,
#           'shuffle': True}

# # Generators
# training_generator = DataGenerator(X_train, y_train, **params)
# validation_generator = DataGenerator(X_val, y_val, **params)

In [0]:
# callbacks
curr_datetime = datetime.now().strftime("%Y%m%d-%H%M%S")

drive_path = '/content/drive/My Drive'
checkpoint_path=  drive_path + '/headline_thaigov_generation/weights/' + curr_datetime + "_weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"


callback_list = [     
      ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min'), 
      EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50),
      # ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=100, min_lr=1e-10),
]

In [0]:
def extractive_classifcation_model():
  inputs = Input(shape=(maxlen, ))
  x = Embedding(vocab_size, maxlen, trainable=True)(inputs)
  x = Bidirectional(LSTM(maxlen, return_sequences=True))(x)
  x = LSTM(maxlen, return_sequences=True)(x)
  outputs = Dense(1, activation='sigmoid')(x)
  model = Model(inputs, outputs)
  model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
  model.summary()
  return Model

In [88]:
extractive_model = extractive_classifcation_model()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 50, 50)            981350    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 50, 100)           40400     
_________________________________________________________________
lstm_7 (LSTM)                (None, 50, 50)            30200     
_________________________________________________________________
dense_3 (Dense)              (None, 50, 1)             51        
Total params: 1,052,001
Trainable params: 1,052,001
Non-trainable params: 0
_________________________________________________________________


In [91]:
extractive_model.fit(X_train, y_train, batch_size=32,  epochs=2000)

AttributeError: ignored

50

In [0]:
# def define_models(n_input=maxlen,n_output=maxlen_output,n_units=32) :
#   #define training encoder model
#   encoder_inputs = Input(shape=(maxlen,))
#   encoder_embedding = Embedding(vocab_size, n_units)(encoder_inputs)
#   encoder  = LSTM(n_units, return_state=True)
#   encoder_outputs, state_h, state_c = encoder(encoder_embedding)
#   encoder_states = [state_h, state_c]

#   # define training decoder model
#   decoder_inputs = Input(shape=(maxlen_output,vocab_size ))
#   # decoder_embedding = Embedding(vocab_size, n_units)
#   # decoder_inputs2 = decoder_embedding(decoder_inputs)
#   decoder_lstm = LSTM(n_units, return_state=True, return_sequences=True)
#   decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

#   # dense_layer = Dense(VOCAB_SIZE, activation='softmax')
#   decoder_dense  = Dense(vocab_size, activation='softmax')
#   decoder_outputs = decoder_dense(decoder_outputs)
#   model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#   #define inferencing encoder model
#   encoder_model = Model(encoder_inputs,encoder_states)
#   #define inference decoder
#   decoder_state_input_h = Input(shape=(n_units,))
#   decoder_state_input_c = Input(shape=(n_units,))
#   decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
#   decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
#   decoder_states = [state_h, state_c]
#   decoder_outputs = decoder_dense(decoder_outputs)
#   decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
#   # return all models
#   return model,encoder_model,decoder_model

In [0]:
# configure problem
# n_features = 50 + 1
n_steps_in = maxlen
n_steps_out = maxlen_output
train, infenc, infdec = define_models(n_units=64)
# train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
train.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])
train.summary()

In [0]:
train.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    epochs=300, callbacks=callback_list)